In [ ]:
# ==========================================
# Task 2-1: 探索的データ分析 (EDA)
# ==========================================

# 1. ライブラリの読み込み
# ------------------------------------------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# 設定
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')
sns.set(font_scale=1.2)
pd.set_option('display.max_columns', None)

# 日本語フォント設定（文字化け回避用）
plt.rcParams['font.family'] = 'sans-serif' 

print("Libraries loaded.")

# 2. データの読み込み
# ------------------------------------------
# ノートブックの実行場所によってパスが変わる可能性があるため、両方に対応
try:
    df = pd.read_csv('../data/simulated_hr_data.csv')
except FileNotFoundError:
    df = pd.read_csv('data/simulated_hr_data.csv')

# データの確認
print(f"Data Shape: {df.shape}")
display(df.head())

# データ型の確認
print("\n--- Data Info ---")
df.info()

# 3. 離職の基本構造 (Overview)
# ------------------------------------------
# 離職フラグの確認（1が離職、0が在籍）
attrition_rate = df['attrition_flag'].mean()
print(f"\n全体の離職発生率（期間合計）: {attrition_rate:.2%}")

# ------------------------------------------------------
# 4. 属性別の離職率比較 (Categorical Features)
# ------------------------------------------------------
categorical_cols = ['job_family', 'gender', 'education']

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for i, col in enumerate(categorical_cols):
    # 平均値（＝離職率）を計算
    prop_df = df.groupby(col)['attrition_flag'].mean().reset_index()
    
    sns.barplot(x=col, y='attrition_flag', data=prop_df, ax=axes[i], palette='Blues_d')
    axes[i].set_title(f'Attrition Rate by {col}')
    axes[i].set_ylabel('Attrition Rate')
    axes[i].set_ylim(0, 0.4) # 比較しやすくY軸を固定
    
    # グラフ上に数値を表示
    for index, row in prop_df.iterrows():
        axes[i].text(index, row.attrition_flag + 0.01, f"{row.attrition_flag:.1%}", 
                     color='black', ha="center")

plt.tight_layout()
plt.show()

# ------------------------------------------------------
# 5. 離職要因の探索 (Numerical Features)
# ------------------------------------------------------
# 離職した人(1)としなかった人(0)で分布はどう違うか？

target_cols = [
    'tenure_months',    # 勤続月数
    'base_salary',      # 給与
    'overtime_hours',   # 残業時間
    'burnout_index',    # 燃え尽き指標
    'engagement_score'  # エンゲージメント
]

fig, axes = plt.subplots(1, 5, figsize=(20, 5))

for i, col in enumerate(target_cols):
    sns.boxplot(x='attrition_flag', y=col, data=df, ax=axes[i], palette='Set2')
    axes[i].set_title(f'{col} vs Attrition')

plt.tight_layout()
plt.show()

# 📝 Insight: 
# 箱ひげ図を見ることで、「離職者は残業が多い傾向にあるか？」「給与が低い傾向にあるか？」を一目で確認できます。

# ------------------------------------------------------
# 6. 相関分析 (Correlation Heatmap)
# ------------------------------------------------------
# 数値列のみ抽出
numeric_df = df.select_dtypes(include=[np.number])
corr = numeric_df.corr()

# 離職フラグとの相関が高い順に表示
print("\n=== Correlation with Attrition Flag ===")
print(corr['attrition_flag'].sort_values(ascending=False))

# ヒートマップ描画
plt.figure(figsize=(12, 10))
sns.heatmap(corr, annot=True, fmt='.2f', cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Heatmap')
plt.show()

# ------------------------------------------------------
# 7. 生存分析的な視点 (Attrition Curve)
# ------------------------------------------------------
# 勤続月数ごとの離職発生数とリスク人口から「離職率」の推移を見る

# 勤続月数ごとの集計
tenure_stats = df.groupby('tenure_months').agg(
    total_count=('employee_id', 'count'),
    attrition_count=('attrition_flag', 'sum')
).reset_index()

plt.figure(figsize=(12, 6))
sns.lineplot(x='tenure_months', y='attrition_count', data=tenure_stats, marker='o', color='crimson')
plt.title('Number of Attritions by Tenure Month (Attrition Curve)')
plt.xlabel('Tenure (Months)')
plt.ylabel('Count of Attritions')
plt.grid(True)
plt.show()

# 「入社直後(オンボーディング期)に山があるか？」「3年目(36ヶ月)に山があるか？」などを確認します。
print("EDA Completed.")